In [131]:
# Libraries

import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline

In [132]:
img = cv2.imread("C:/Users/vidit/Documents/UW_Research_Mittal_Lab/Pipeline_Test_4_4_22/Individual_Clusters/19/required_cluster.jpg")

In [133]:
img.shape

(2588, 4706, 3)

In [134]:
img_res = img.reshape((-1, 3))
img_res = np.float32(img2)

h = np.linspace(0, img.shape[0] - 1, img.shape[0])
w = np.linspace(0, img.shape[1] - 1, img.shape[1])

w, h = np.meshgrid(w, h)
w = w.reshape(-1, 1)
h = h.reshape(-1, 1)

In [135]:
# Normalization

img_res_1 = img_res/255.
w_1 = 0.15 * w/4705.
h_1 = 0.15 * h/2587.

img_mod_1 = np.concatenate((img_res_1, h_1, w_1), axis=1)
img_mod_1 = np.float32(img_mod_1)

In [136]:
# We define criteria based on epsilon and number of iterations i.e. whichever is achieved first
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TermCriteria_MAX_ITER, 10, 0.1)
# Number of clusters
k = 4
# Number of attempts (no. of times algo is executed using different labellings)
attempts = 10

compact, label, center = cv2.kmeans(img_mod_1, k, None, criteria, attempts, cv2.KMEANS_RANDOM_CENTERS)

In [137]:
center

array([[0.23809248, 0.1958109 , 0.31422406, 0.06906977, 0.06798584],
       [0.9280577 , 0.9204676 , 0.9424032 , 0.06352073, 0.0734175 ],
       [0.99995804, 0.99991465, 0.99996156, 0.07363418, 0.07413188],
       [0.16962898, 0.12439784, 0.25162253, 0.06428799, 0.07880627]],
      dtype=float32)

In [138]:
center_1 = np.delete(center, [3, 4], axis=1)*255
center_1 = np.uint8(center_1)
center_1

array([[ 60,  49,  80],
       [236, 234, 240],
       [254, 254, 254],
       [ 43,  31,  64]], dtype=uint8)

In [139]:
colors = [np.array([204, 0, 0]), np.array([204, 204, 0]), np.array([0, 204, 0]), 
          np.array([0, 153, 153]), np.array([102, 0, 204]), np.array([102, 0, 51]),
         np.array([51, 153, 255])] 

In [151]:
for cluster_num in tqdm(range(4)):
        new_center = np.zeros(center_1.shape)
        for i in range(len(center_1)):
            if i == cluster_num:
                col = np.random.randint(0, 6)
                new_center[i] = colors[col]
            else:
                new_center[i] = np.array([255, 255, 255])

        new_center = np.uint8(new_center)
        df = pd.DataFrame(label)
        df[0] = df[0].apply(lambda x: cluster_num if x==cluster_num else cluster_num-1)
        lab_new = np.array(df)
        new_res = new_center[lab_new.flatten()]
        new_res = new_res.reshape((img.shape))
        plt.imsave("C:/Users/vidit/Documents/UW_Research_Mittal_Lab/Pipeline_Test_4_4_22/Individual_Clusters/19/Spatial/" +
                   str(cluster_num) + ".jpg", new_res)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:26<00:00,  6.55s/it]


In [156]:
def merge_2_clusters(n1, n2, img):
    
    img_new = img.reshape((-1, 3))
    img_req = np.zeros(img_new.shape)
    
    for i in tqdm(range(len(label))):
        if label[i] == n1:
            img_req[i] = colors[5]
        elif label[i] == n2:
            img_req[i] = colors[2]
        else:
            img_req[i] = np.array([255, 255, 255])
            
    plt.imsave("C:/Users/vidit/Documents/UW_Research_Mittal_Lab/Pipeline_Test_4_4_22/Individual_Clusters/19/Spatial/merged_cluster.jpg", 
               np.uint8(img_req.reshape(img.shape)))
    
    return None

In [157]:
merge_2_clusters(0, 3, img)

100%|██████████████████████████████████████████████████████████████████| 12179128/12179128 [01:04<00:00, 189404.77it/s]


In [148]:
mer_img = cv2.imread("Test_Extracted.tif")

overlay_img = cv2.addWeighted(img, 0.6, mer_img, 0.4, 0)

plt.imsave("C:/Users/vidit/Documents/UW_Research_Mittal_Lab/Pipeline_Test_4_4_22/Individual_Clusters/19/Spatial/overlay_image.jpg",
           overlay_img)